In [28]:
import random
import data
import os
import parameters as p
import tensorflow as tf
import importlib

In [29]:
importlib.reload(p)
importlib.reload(data)

<module 'data' from 'C:\\Users\\user\\PycharmProjects\\nuty5\\jupyter\\data.py'>

In [30]:
random.seed()
classes = data.retrieve_class_names()
output_dim = p.ELEMENTS_MAX_NUMBER * (5 + len(classes))
pages = data.Retriever().retrieve(classes)

5% of the files processed, we are at 100 in 1864.
11% of the files processed, we are at 200 in 1864.
16% of the files processed, we are at 300 in 1864.
21% of the files processed, we are at 400 in 1864.
27% of the files processed, we are at 500 in 1864.
32% of the files processed, we are at 600 in 1864.
38% of the files processed, we are at 700 in 1864.
43% of the files processed, we are at 800 in 1864.
48% of the files processed, we are at 900 in 1864.
54% of the files processed, we are at 1000 in 1864.
59% of the files processed, we are at 1100 in 1864.
64% of the files processed, we are at 1200 in 1864.
70% of the files processed, we are at 1300 in 1864.
75% of the files processed, we are at 1400 in 1864.
80% of the files processed, we are at 1500 in 1864.
86% of the files processed, we are at 1600 in 1864.
91% of the files processed, we are at 1700 in 1864.
97% of the files processed, we are at 1800 in 1864.


In [31]:
files = os.listdir(p.PNG_PATH)
generator = data.Generator(pages, output_dim, classes)
ds = tf.data.Dataset.from_generator(
    generator.generator,
    args=[files],
    output_types=(tf.int32, tf.float32),
    output_shapes=([p.BATCH_SIZE, p.Y, p.X, 3], [p.BATCH_SIZE, output_dim]))

In [32]:
from keras import backend as K

def get_loss(y_actual,y_pred):

    y_actual_reshaped = K.reshape(y_actual, [p.BATCH_SIZE * p.ELEMENTS_MAX_NUMBER, -1])
    y_pred_reshaped = K.reshape(y_pred, [p.BATCH_SIZE * p.ELEMENTS_MAX_NUMBER, -1])

    main_probs_act = y_actual_reshaped[..., :1]
    main_probs_pred = y_pred_reshaped[..., :1]
    bb_probs_act = y_actual_reshaped[..., 1:5]
    bb_probs_pred =  y_pred_reshaped[..., 1:5]
    classes_probs_act = y_actual_reshaped[..., 5:]
    classes_probs_pred =  y_pred_reshaped[..., 5:]

    loss = K.sum(K.binary_crossentropy(main_probs_act, main_probs_pred, from_logits=True))
    loss += K.sum(K.sum(K.abs(bb_probs_act - bb_probs_pred), 1) * main_probs_act)
    loss += K.sum(K.sum(K.abs(classes_probs_act - classes_probs_pred), 1) * main_probs_act)

    return loss


In [33]:
top = tf.keras.models.Sequential([
    # tf.keras.layers.Conv2D(16, (3,3), activation='relu', input_shape=(p.Y,p.X,1)),
    # tf.keras.layers.MaxPooling2D(2,2),
    # tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(1024, activation='relu'),
    tf.keras.layers.Dense(1024, activation='relu')
])

vgg = tf.keras.applications.vgg19.VGG19(
    include_top=False, weights='imagenet',
    input_shape=(p.Y,p.X,3), pooling='avg'
)

model = tf.keras.Sequential(vgg.layers)
model.add(tf.keras.layers.Dense(output_dim, activation='relu'))
#model.summary()

In [ ]:
model.compile(optimizer='adam',
              loss=get_loss,
              metrics=tf.keras.metrics.MeanAbsoluteError(name="mean_absolute_error", dtype=None))
a = model. fit(ds, epochs=1)

      9/Unknown - 151s 17s/step - loss: 364.8649 - mean_absolute_error: 6965096.5000

In [ ]:
import numpy as np
import imageio as im
# image = np.array(im.imread(r'C:\Users\user\Desktop\ex2.png'))
# pred = model.predict(np.reshape(image[0:p.X,0:p.X,0:3] / 255, (1, p.Y, p.X, 3)))
# pred *=p.X

aa = generator.generator(files).__next__()
aaa = model.predict(aa[0])
#print(aa[1],aaa )
np.sum(np.abs(aaa - aa[1]))

In [ ]:
a = aa[0][0]
import viewer as v
v.show_prediction(aa[1][0], aa[0][0])
